## Sentiment Analysis of Movie Reviews

**Objective**: Build a sentiment analysis model to classify IMDb movie reviews as positive or negative.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pip install nltk


There was an issue accessing the IMDb movie review dataset. We used an alternative method to load the dataset `nltk` library, which provides access to the IMDb movie reviews dataset.

## Data Preparation 

In [11]:
import nltk
from nltk.corpus import movie_reviews

# Download the IMDb movie reviews dataset
nltk.download('movie_reviews')

# Load the movie reviews
reviews = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        review = {
            'text': movie_reviews.raw(fileid),
            'label': category
        }
        reviews.append(review)

# Convert to DataFrame
imdb_df = pd.DataFrame(reviews)

# Display the first few rows of the dataset
print(imdb_df.head())

# Check the distribution of labels
print(imdb_df['label'].value_counts())


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
                                                text label
0  plot : two teen couples go to a church party ,...   neg
1  the happy bastard's quick movie review \ndamn ...   neg
2  it is movies like these that make a jaded movi...   neg
3   " quest for camelot " is warner bros . ' firs...   neg
4  synopsis : a mentally unstable man undergoing ...   neg
label
neg    1000
pos    1000
Name: count, dtype: int64


## Text Preprocessing 

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

# Download stopwords
nltk.download('stopwords')

# Initialize stemmer and set of stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing special characters and stopwords, and performing stemming
    preprocessed_tokens = []
    for token in tokens:
        if token not in string.punctuation and token not in stop_words:
            preprocessed_tokens.append(stemmer.stem(token))
    
    # Join tokens back into text
    preprocessed_text = ' '.join(preprocessed_tokens)
    
    return preprocessed_text

# Apply text preprocessing to the 'text' column of the DataFrame
imdb_df['preprocessed_text'] = imdb_df['text'].apply(preprocess_text)

# Display the preprocessed text
print(imdb_df['preprocessed_text'].head())


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0    plot two teen coupl go church parti drink driv...
1    happi bastard 's quick movi review damn y2k bu...
2    movi like make jade movi viewer thank invent t...
3    `` quest camelot `` warner bro first feature-l...
4    synopsi mental unstabl man undergo psychothera...
Name: preprocessed_text, dtype: object


## TF-IDF Feature Extraction 

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed text data
X_tfidf = tfidf_vectorizer.fit_transform(imdb_df['preprocessed_text'])

# Display the shape of the TF-IDF matrix
print("TF-IDF matrix shape:", X_tfidf.shape)

TF-IDF matrix shape: (2000, 27273)


## Model Selection 

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, imdb_df['label'], test_size=0.2, random_state=42)

# Initialize the logistic regression model
logistic_regression_model = LogisticRegression()

# Train the model on the training data
logistic_regression_model.fit(X_train, y_train)


LogisticRegression()

## Model Evaluation 

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred = logistic_regression_model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='pos')
recall = recall_score(y_test, y_pred, pos_label='pos')
f1 = f1_score(y_test, y_pred, pos_label='pos')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


Accuracy: 0.805
Precision: 0.8
Recall: 0.8159203980099502
F1-Score: 0.8078817733990147


## Model Tuning (Optional)

In [18]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}

# Initialize the grid search with cross-validation
grid_search = GridSearchCV(estimator=logistic_regression_model, param_grid=param_grid, cv=5)

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluate the model with the best hyperparameters
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best, pos_label='pos')
recall_best = recall_score(y_test, y_pred_best, pos_label='pos')
f1_best = f1_score(y_test, y_pred_best, pos_label='pos')

# Print the evaluation metrics for the best model
print("Accuracy (Best Model):", accuracy_best)
print("Precision (Best Model):", precision_best)
print("Recall (Best Model):", recall_best)
print("F1-Score (Best Model):", f1_best)


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

Best Hyperparameters: {'C': 10, 'penalty': 'l2'}
Accuracy (Best Model): 0.8375
Precision (Best Model): 0.8366336633663366
Recall (Best Model): 0.8407960199004975
F1-Score (Best Model): 0.8387096774193548


There were some failures during the grid search process, likely due to incompatible hyperparameter combinations. However, despite these failures, we still managed to find the best hyperparameters for the logistic regression model.

Here's the result of the hyperparameter tuning:

**Best Hyperparameters: {'C': 10, 'penalty': 'l2'}
Accuracy (Best Model): 0.8375
Precision (Best Model): 0.8366
Recall (Best Model): 0.8408
F1-Score (Best Model): 0.8387**

## Prediction 

In [19]:
# Example text for prediction
new_reviews = [
    "This movie was amazing! I loved every moment of it.",
    "The acting was terrible and the plot was boring.",
    "I couldn't stop laughing throughout the entire film."
]

# Preprocess the new reviews
preprocessed_new_reviews = [preprocess_text(review) for review in new_reviews]

# Convert the preprocessed text into TF-IDF features
X_new_tfidf = tfidf_vectorizer.transform(preprocessed_new_reviews)

# Make predictions on the new reviews
predictions = logistic_regression_model.predict(X_new_tfidf)

# Print the predictions
for review, prediction in zip(new_reviews, predictions):
    print("Review:", review)
    print("Predicted Sentiment:", prediction)
    print()


Review: This movie was amazing! I loved every moment of it.
Predicted Sentiment: pos

Review: The acting was terrible and the plot was boring.
Predicted Sentiment: neg

Review: I couldn't stop laughing throughout the entire film.
Predicted Sentiment: pos



## Conclusion 

After completing the sentiment analysis of IMDb movie reviews using a logistic regression model, we achieved the following results:

The logistic regression model achieved an accuracy of approximately 83.75% on the test data, indicating that it can effectively classify movie reviews as positive or negative.
The precision, recall, and F1-score of the model were around 0.84, indicating a good balance between precision (ability to avoid false positives) and recall (ability to identify positive samples).
Insights from the analysis:

The logistic regression model, trained on TF-IDF features, performed well in classifying movie reviews as positive or negative, demonstrating the effectiveness of text classification techniques in sentiment analysis tasks.
The model's performance could potentially be further improved with additional preprocessing steps, feature engineering, or experimenting with different machine learning algorithms.

Overall, the sentiment analysis experiment provided valuable insights into the sentiment of IMDb movie reviews and demonstrated the effectiveness of machine learning models in analyzing text data.